In [7]:
import numpy as np
import os
import sys
from resource import *
import time
import psutil

In [8]:
input_file = open("./SampleTestCases/input2.txt")

In [9]:
input_file

<_io.TextIOWrapper name='./SampleTestCases/input2.txt' mode='r' encoding='UTF-8'>

In [10]:
seq = input_file.read().split("\n")

In [11]:
seq
strings = []
string = ''
i = 0

In [12]:
for s in seq:
    if not s.isdigit(): 
        if string == '': 
            string = s
            base_len = len(s)
        else: 
            if base_len * 2 ** i != len(string): 
                print("Error for string length.")
                sys.exit(1)
            strings.append(string)
            string = s
            i = 0
            base_len = len(s)
    else: 
        i += 1
        s = int(s)
        string = string[0:s+1] + string + string[s+1:]
if base_len * 2 ** i != len(string): 
    print("Error for string length.")
    sys.exit(1)
strings.append(string)

In [13]:
strings

['ACACACTGACTACTGACTGGTGACTACTGACTGGACTGACTACTGACTGGTGACTACTGACTGG',
 'TTATTATACGCGACGCGATTATACGCGACGCG']

In [14]:
string1 = strings[0]
string2 = strings[1]
print(string1, string2)

ACACACTGACTACTGACTGGTGACTACTGACTGGACTGACTACTGACTGGTGACTACTGACTGG TTATTATACGCGACGCGATTATACGCGACGCG


In [15]:
code_dic = {"A": 1, "C": 2, "G": 3, "T": 5}
alpha = [0] * 26
alpha[2], alpha[3], alpha[5], alpha[6], alpha[10], alpha[15] = 110, 48, 94, 118, 48, 110
delta = 30

In [16]:
dp = np.zeros((len(string1) + 1, len(string2) + 1))
for i in range(dp.shape[0]): 
    for j in range(dp.shape[1]): 
        if i == 0: 
            dp[i, j] = j * delta
            continue
        if j == 0:
            dp[i, j] = i * delta
            continue
        dp[i, j] = min(dp[i - 1, j - 1] + alpha[code_dic[string1[i - 1]] * code_dic[string2[j - 1]]], 
                       dp[i - 1, j] + delta, 
                       dp[i, j - 1] + delta)
cost = int(dp[i, j])
aligned1_back = ''
aligned2_back = ''    
while (i != 0) or (j != 0):
    if i == 0: 
        aligned1_back += '_'
        aligned2_back += string2[j - 1]
        j -= 1
        continue
    if j == 0: 
        aligned1_back += string1[i - 1]
        aligned2_back += '_'
        i -= 1
        continue
    diag_value = dp[i - 1, j - 1] + alpha[code_dic[string1[i - 1]] * code_dic[string2[j - 1]]]
    if diag_value == dp[i, j]: 
        aligned1_back += string1[i - 1]
        aligned2_back += string2[j - 1]
        i -= 1
        j -= 1
        continue
    if dp[i - 1, j] + delta == dp[i, j]: 
        aligned1_back += string1[i - 1]
        aligned2_back += '_'
        i -= 1
        continue
    if dp[i, j - 1] + delta == dp[i, j]: 
        aligned1_back += "_"
        aligned2_back += string2[j - 1]
        j -= 1
aligned1 = aligned1_back[::-1]
aligned2 = aligned2_back[::-1]


In [17]:
dp

array([[   0.,   30.,   60., ...,  900.,  930.,  960.],
       [  30.,   60.,   90., ...,  870.,  900.,  930.],
       [  60.,   78.,  108., ...,  840.,  870.,  900.],
       ...,
       [1860., 1830., 1800., ..., 1128., 1122., 1140.],
       [1890., 1860., 1830., ..., 1098., 1128., 1122.],
       [1920., 1890., 1860., ..., 1128., 1158., 1128.]])

In [18]:
aligned1

'ACACACTGACTACTGACTGGTGACTACTGACTGGACTGACTACTGACTGGTGACTAC_TGACTG_G'

In [19]:
aligned2

'__________T__T_A_T__T_A_TAC_G_C__GAC_G____C_GA_T__T_A_TACGCGAC_GCG'

In [20]:
len(string2)

32

In [21]:
print(dp[1:6, 1:6])

[[ 60.  90.  60.  90. 120.]
 [ 78. 108.  90. 108. 138.]
 [108. 138. 108. 138. 168.]
 [138. 156. 138. 156. 186.]
 [168. 186. 156. 186. 216.]]


In [22]:
os.getpid()

24287

In [23]:
process = psutil.Process()
memory_info = process.memory_info()
memory_consumed = int(memory_info.rss/1024)
print(memory_info, memory_consumed)

pmem(rss=108453888, vms=36525211648, pfaults=67143, pageins=1490) 105912


In [24]:
memory_consumed

105912

In [25]:
process = psutil.Process(os.getpid())
memory_info = process.memory_info()
memory_consumed = int(memory_info.rss/1024)

In [26]:
info = process.memory_full_info()

In [27]:
info.rss

108462080

In [28]:
memory_info.rss

108457984

In [29]:
code_dic = {"A": 1, "C": 2, "G": 3, "T": 5}
alpha = [0] * 26
alpha[2], alpha[3], alpha[5], alpha[6], alpha[10], alpha[15] = 110, 48, 94, 118, 48, 110
delta = 30

In [30]:
def alignment_efficient(string1, string2): 
    if len(string1) < 3 or len(string2) < 3: 
        dp = np.zeros((2, 2))
        for i in range(dp.shape[0]):  
            for j in range(dp.shape[1]): 
                if i == 0: # initialize 
                    dp[i, j] = j * delta
                    continue
                if j == 0: # initialize 
                    dp[i, j] = i * delta
                    continue
                dp[i, j] = min(dp[i - 1, j - 1] + alpha[code_dic[string1[i - 1]] * code_dic[string2[j - 1]]], # DP step
                            dp[i - 1, j] + delta, 
                            dp[i, j - 1] + delta)
        cost = int(dp[i, j])
        aligned1_back = ''
        aligned2_back = '' 
        while (i != 0) or (j != 0):
            if i == 0: # when string2 is longer
                aligned1_back += '_'
                aligned2_back += string2[j - 1]
                j -= 1
                continue
            if j == 0: # when string1 is longer
                aligned1_back += string1[i - 1]
                aligned2_back += '_'
                i -= 1
                continue
            diag_value = dp[i - 1, j - 1] + alpha[code_dic[string1[i - 1]] * code_dic[string2[j - 1]]] # traceback diagonal
            if diag_value == dp[i, j]: 
                aligned1_back += string1[i - 1]
                aligned2_back += string2[j - 1]
                i -= 1
                j -= 1
                continue
            if dp[i - 1, j] + delta == dp[i, j]: # traceback along string1
                aligned1_back += string1[i - 1]
                aligned2_back += '_'
                i -= 1
                continue
            if dp[i, j - 1] + delta == dp[i, j]: #traceback along string2
                aligned1_back += "_"
                aligned2_back += string2[j - 1]
                j -= 1
        aligned1 = aligned1_back[::-1] # get aligned string1
        aligned2 = aligned2_back[::-1] # get aligned string2
        return cost, aligned1, aligned2
    dp_l = np.zeros((2, len(string2) + 1))
    dp_r = np.zeros((2, len(string2) + 1))
    for i in range(len(string1) // 2 + 1): 
        for j in range(dp_l.shape[1]): 
            if i == 0:
                dp_l[i, j] = j * delta
                continue
            if j == 0:
                dp_l[1, 0] = i * delta
                continue
            dp_l[1, j] = min(dp_l[0, j - 1] + alpha[code_dic[string1[i - 1]] * code_dic[string2[j - 1]]], 
                            dp_l[0, j] + delta, 
                            dp_l[1, j - 1] + delta)
        for j in range(dp_l.shape[1]): 
            dp_l[0, j] = dp_l[1, j]
    for i in range(len(string1) - (len(string1) // 2) + 1): 
        for j in range(dp_r.shape[1]): 
            if i == 0: 
                dp_r[i, j] = j * delta
                continue
            if j == 0: 
                dp_r[1, 0] = i * delta
                continue
            dp_r[1, j] = min(dp_r[0, j - 1] + alpha[code_dic[string1[len(string1) - i]] * code_dic[string2[len(string2) - j]]], 
                            dp_r[0, j] + delta, 
                            dp_r[1, j - 1] + delta)
            for j in range(dp_l.shape[1]): 
                dp_r[0, j] = dp_r[1, j]
    cut_point = 0
    for j in range(dp_r.shape[1]): 
        if dp_l[0, j] + dp_r[0, len(string2) - j] < dp_l[0, cut_point] + dp_r[0, len(string2) - cut_point]: 
            cut_point = j
    #cost = dp_l[0, cut_point] + dp_r[0, len(string2) - cut_point]
    cost1, aligned1_l, aligned2_l = alignment_efficient(string1[:(len(string1) // 2)], string2[:cut_point])
    cost2, aligned1_r, aligned2_r = alignment_efficient(string1[(len(string1) // 2):], string2[cut_point:])
    cost = cost1 + cost2
    return cost, aligned1_l + aligned1_r, aligned2_l + aligned2_r

In [31]:
cost, al1, al2 = alignment_efficient(string1, string2)

In [35]:
print(string1, string2)

ACACACTGACTACTGACTGGTGACTACTGACTGGACTGACTACTGACTGGTGACTACTGACTGG TTATTATACGCGACGCGATTATACGCGACGCG


In [33]:
al1

'_AAA_A_CC_G_T_TGA_TTG'

In [34]:
al2

'T_GGC_G_TT_A_G_GAG_CG'

In [420]:
dp_l = np.zeros((2, len(string2) + 1))
dp_r = np.zeros((2, len(string2) + 1))
for i in range(len(string1) // 2 + 1): 
    for j in range(dp_l.shape[0]): 
        if i == 0:
            dp_l[i, j] = j * delta
            continue
        if j == 0:
            dp_l[1, 0] = i * delta
            continue
        dp_l[1, j] = min(dp_l[0, j - 1] + alpha[code_dic[string1[i - 1]] * code_dic[string2[j - 1]]], 
                         dp_l[0, j] + delta, 
                         dp_l[1, j - 1] + delta)
    for j in range(dp_l.shape[0]): 
        dp_l[0, j] = dp_l[1, j]
for i in range(len(string1) - (len(string1) // 2) + 1): 
    for j in range(dp_r.shape[0]): 
        if i == 0: 
            dp_r[i, j] = j * delta
            continue
        if j == 0: 
            dp_r[1, 0] = i * delta
            continue
        dp_r[1, j] = min(dp_r[0, j - 1] + alpha[code_dic[string1[len(string1) - i]] * code_dic[string2[len(string2) - j]]], 
                         dp_r[0, j] + delta, 
                         dp_r[1, j - 1] + delta)
        for j in range(dp_l.shape[0]): 
            dp_r[0, j] = dp_r[1, j]
cut_point = 0
for j in range(dp_r.shape[0]): 
    if dp_l[0, j] + dp_r[0, len(string2) - j] < dp_l[0, cut_point] + dp_r[0, len(string2) - cut_point]: 
        cut_point = j
aligned1_l, aligned2_l = alignment_efficient(string1[:(len(string1) // 2)], string2[:cutpoint])
aligned1_r, aligned2_r = alignment_efficient(string1[(len(string1) // 2):], string2[cutpoint:])


NameError: name 'alignment_efficient' is not defined

In [415]:
print(i, j)

0 2


In [416]:
range(dp_l.shape[0])

range(0, 33)

In [417]:
dp_l.shape

(33, 2)